In [23]:
import numpy as np
from scipy.stats import unitary_group
from numpy import linalg as LA
from scipy.linalg import block_diag
#from scipy.linalg import logm, expm
import qiskit

In [42]:
d=5
ws=[np.array([0.1,0.3,0.4,0.8/3,0.8/3]),np.array([0.4,0.4,0.4,0.4,0.4])]
#print(gamma0s)

In [30]:
def h(xx):
    x=np.real(xx)
    return -x*np.log(x)-(1-x)*np.log(1-x)
def H(v):
    summ=0
    for j in range(d):
        summ=summ+h(v[j])
    return summ

#the following functions entail conjectures

#def H1(w):
#    v=...
#    return H(v)

def H2(w):
    appo=(w[0]+w[3]+w[4])/3
    appo2=(w[1]+w[2])/2
    v=np.array([appo,appo,appo,appo2,appo2])
    return H(v)
def H3(w):
    appo=(w[0]+w[1]+w[4])/3
    appo2=(w[2]+w[3])/2
    v=np.array([appo,appo,appo,appo2,appo2])
    return H(v)

#the following regime might never be there
def H4(w):
    appo=(w[0]+w[2]+w[4])/3
    appo2=(w[1]+w[3])/2
    v=np.array([appo,appo,appo,appo2,appo2])
    return H(v)

#the following fixes w[2]=w_3 and maximally mixes the other entries when possible (see (2 2 1) rules),
#and returns 0 otherwise 
#def H5(w):
#    t=sum(w)
#    if ...:
#        return(0)
#    else:
#        v=...
#        return H(v)

#the following fixes w[1]=w_2 and maximally mixes the other entries when possible (see (2 2 1) rules),
#and returns 0 otherwise 
#......
#before coding this check if it can ever happen!


def Hmax(w):
    return max([H2(w),H3(w),H4(w)])
    

In [43]:
#now do numerical maximization to find Imax(w) values

#number of unitaries
n=100000

traces=[sum(ws[j]) for j in range(len(ws))]
gamma0s=[np.diag(ws[j]) for j in range(len(ws))]
maxbounds=[d*h(sum(ws[j])/d) for j in range(len(ws))]

vbests=[ws[j] for j in range(len(ws))]
Imaxs=[0 for i in range(len(ws))]

#prob better to allocate memory now...
gamma=np.identity(d,dtype=complex)
U=np.identity(d,dtype=complex)
Udag=np.identity(d,dtype=complex)
v=np.zeros(d,dtype=float)
I=0.
block1=np.identity(2,dtype=complex)
block2=np.identity(2,dtype=complex)
block1ev=np.zeros(2,dtype=complex)
block2ev=np.zeros(2,dtype=complex)

for j in range(n):
    U=unitary_group.rvs(d)
    Udag=np.transpose(np.conjugate(U))
    #treat all vectors with the same unitary!
    for k in range(len(ws)):
        gamma=np.matmul(U,np.matmul(gamma0s[k],Udag))
    #print(gamma.round(2))
        block1=gamma[0:2,0:2]
        block2=gamma[2:4,2:4]
        block1ev=LA.eigvals(block1)
        block2ev=LA.eigvals(block2)
        v[0]=np.real(block1ev[0])
        v[1]=np.real(block1ev[1])
        v[2]=np.real(block2ev[0])
        v[3]=np.real(block2ev[1])
        v[4]=np.real(gamma[4][4])
        I=H(v)
        if I>Imaxs[k]:
            for l in range(d):
                vbests[k][l]=v[l]
            Imaxs[k]=I
for k in range(len(ws)):
    vbests[k]=np.sort(vbests[k])

In [38]:
#Now analysis of conjectures
#for k in range(len(ws))
#    if H1(ws[k])>Imaxs[k]
#        print(...)

array([0.4, 0.4, 0.4, 0.4, 0.4])

In [39]:
vbests[0]


array([0.25527328, 0.26031281, 0.26557989, 0.27358055, 0.27858679])

In [40]:
Imaxs

[2.898652595281461, 3.365058335046283]

In [46]:
maxbounds[0]-Imaxs[0]

0.001313947230318302